### QR Analyzer
Developed by: Miguel de la Cal Bravo & Félix Paniagua Mérida

#### Install and import dependencies
You will need to install the following packages to use this tool:

In [ ]:
#!pip install opencv-python nest-asyncio pyfiglet vt-py scikit-learn nltk

In [ ]:
from pyfiglet import Figlet
import cv2
import vt
import nest_asyncio
nest_asyncio.apply()
import os
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
import re
import logging
logging.getLogger("sklearn.utils.testing").setLevel(logging.WARNING)
import warnings  
warnings.filterwarnings('ignore')

#### Analysis with VirusTotal API
This function is used to analyze a URL detected from a QR code with VirusTotal API.

The URL will be reported as malware if three or more engines detect it as malicious or suspicious.

You can define your own functions to analyze a URL with different tools.

_NOTE: To use this feature you will need Internet connection and set your $VT_API environment variable to your API Key._

If the analysis reports that 3 or more antivirus engines have detected that the website as _malicious_ or _suspicious_, then a _"[!!] PHISING/MALWARE"_ message will be printed on screen in red. Otherwise, the website will be assumed to be safe and a _"[OK] LEGITIMATE"_ message will be displayed in green.

In [ ]:
def analyze_vt(data, analyzed_urls):
    
    # Set API Key from $VT_API env variable
    client = vt.Client(os.environ.get("VT_API"))
    
    # Analyze QR Code URL
    url_id = vt.url_id(str(data))
    url = client.get_object("/urls/{}", url_id)
    
    # If reported by < 3 analyzers as malicious and suspicious ~> LEGITIMATE
    if url.last_analysis_stats['malicious'] < 3 and url.last_analysis_stats['suspicious'] < 3:
        print("[OK] Website is LEGITIMATE\n")
        analyzed_urls[data] = "false"
        cv2.putText(img, "[OK] LEGITIMATE", (int(bbox[0][0][0]), int(bbox[0][0][1]) - 60), cv2.FONT_HERSHEY_DUPLEX,
                1, (0, 255, 0), 2)
    # If reported by >= 3 analyzers as malicious or suspicious ~> MALWARE  
    else:
        print("[!!] Website is PHISING or contains MALWARE\n")
        analyzed_urls[data] = "true"
        cv2.putText(img, "[!!] PHISING/MALWARE", (int(bbox[0][0][0]), int(bbox[0][0][1]) - 60), cv2.FONT_HERSHEY_DUPLEX,
                1, (0, 0, 255), 2)

#### Analysis with Machine Learning models
In this function we will use Logistic Regression to analyze a URL detected from a QR code.

The models have been trained previously and will be loaded as PKL files. You can find the training scripts in: https://github.com/filip7575/morteruelo

_NOTE: To use this feature you do not need Internet connection._

In [ ]:
# Analyze with ML (Logistic Regression)
def analyze_ml_lr(data, analyzed_urls):
    
    qr_url=[str(re.sub(r"^https?://", "", data))]
    pkl_file = "Pickle_type_Model_LM.pkl"
    loaded_mo = pickle.load(open(pkl_file, 'rb'))
    results = loaded_mo.predict(qr_url)

    # If URL is detected as 'good' ~> LEGITIMATE
    if results[0] == 'good':
        print("[OK] Website is LEGITIMATE\n")
        analyzed_urls[data] = "false"
        cv2.putText(img, "[OK] LEGITIMATE", (int(bbox[0][0][0]), int(bbox[0][0][1]) - 60), cv2.FONT_HERSHEY_DUPLEX,
                1, (0, 255, 0), 2)
    # URL detected as 'bad' ~> MALWARE
    else:
        print("[!!] Website is PHISING or contains MALWARE\n")
        analyzed_urls[data] = "true"
        cv2.putText(img, "[!!] PHISING/MALWARE", (int(bbox[0][0][0]), int(bbox[0][0][1]) - 60), cv2.FONT_HERSHEY_DUPLEX,
                1, (0, 0, 255), 2)

#### Analyzing QR codes with your camera
Once defined the functions that will analyze the URL detected from a QR code, we will configure the camera and functions needed to scan the QR code.

After that, we will define a set of keys to enter and operate the program, when a QR code has been detected:`

- <kbd>c</kbd> : Analyze with VirusTotal API
- <kbd>l</kbd> : Analyze with ML (Logistic Regression)
- <kbd>q</kbd> : Close the camera and exit the program

When a URL is detected as legitimate a message will be printed on screen in green. Otherwise, the website will be assumed to be safe and a

If you have more than one camera connected to your device, please check ***cap = cv2.VideoCapture(0)*** line and change ***0*** to other value to select your camera. Remember to use ```ls /dev``` command to check video capture devices connected to the machine.

In [ ]:
# Camera configurations, 750x563p, 30 FPS
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 750)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 563)
cap.set(cv2.CAP_PROP_FPS, 30)

In [ ]:
# QR code detection object
detector = cv2.QRCodeDetector()

# Variables
data_aux = ""       # previous QR detected
analyzed_urls = {}  # previous analyzed URLs
f = Figlet(font='big')
print(f.renderText('QR Analyzer'))

while True:

    # Get the image and bounding coords and data
    _, img = cap.read()
    data, bbox, _ = detector.detectAndDecode(img)

    # If there is a bounding box, draw one, along with the data
    if(bbox is not None):
        for i in range(len(bbox)):
            cv2.line(img, tuple(bbox[i][0].astype(int)), tuple((bbox[(i+1) % len(bbox)][0]).astype(int)), color=(255,
                     0, 255), thickness=2)
        cv2.putText(img, data, (int(bbox[0][0][0]), int(bbox[0][0][1]) - 10), cv2.FONT_HERSHEY_DUPLEX,
                    1.25, (0, 242, 242), 2)
              
        # Check if a QR Code is detected
        if data and data_aux != data and data not in analyzed_urls:
            print()
            print("[+] QR Code detected! ", data)
            print("[+] Press `c` key to analyze website with VT API\n" +
                  "[+] Press `l` key to analyze website with ML (Logistic Regression)")
            data_aux = data

        # Wait for key pressed
        key_pressed = cv2.waitKey(1)

        if key_pressed != -1:
            
            # Press `c` key
            if key_pressed == 99:
                try:
                    print(f"[+] Analyzing {data} with VT API")
                    analyze_vt(data, analyzed_urls)
                except Exception as e:
                    print(f"[!] ERROR: Exception {type(e).__name__}")
                    continue
                    
            # Press `l` key
            elif key_pressed == 108:
                print(f"[+] Analyzing {data} with ML (Logistic Regression)")
                analyze_ml_lr(data, analyzed_urls)
                
            # Press `q` key
            elif key_pressed == 113:
                print("[!] EXITTING...")
                break
            
        # If website was analyzed, print previous result
        if data in analyzed_urls:
            if analyzed_urls[data] == "false":
                cv2.putText(img, "[OK] LEGITIMATE", (int(bbox[0][0][0]), int(bbox[0][0][1]) - 60), cv2.FONT_HERSHEY_DUPLEX,
                        1, (0, 255, 0), 2)
            if analyzed_urls[data] == "true":
                cv2.putText(img, "[!!] PHISING/MALWARE", (int(bbox[0][0][0]), int(bbox[0][0][1]) - 60), cv2.FONT_HERSHEY_DUPLEX,
                        1, (0, 0, 255), 2)              
                        
    # Display the image preview
    cv2.imshow("code detector", img)
    
    # Exit
    if(cv2.waitKey(1) == ord("q")):
        print("[!] EXITTING...")
        break
        

# Free camera object and exit
cap.release()
cv2.destroyAllWindows()